   
   - 1- Gravitational allocation
   - 2- Recursive repuslion of RPPP


In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format='retina'
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

# make plots look nice
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 12
plt.rc('axes', labelsize=22)
plt.rc('legend', fontsize=12)
#formating ticks
plt.rcParams['axes.formatter.limits'] = (-1, 1)
plt.rcParams['axes.formatter.useoffset'] = False
plt.rcParams['text.usetex'] = True

In [ ]:
import math
import numpy as np
import time
import pickle

# 1- Gravitational allocation 2D

In [ ]:
from mcrppy.spatial_windows import BallWindow
from mcrppy.point_processes import HomogeneousPoissonPointProcess
from mcrppy.repelled_point_process import epsilon_critical
from mcrppy.coulomb_force import force_base

In [ ]:
def starting_points(point, intenisty, nb_output=40, scale=0.02):
    # in 2d
    point = np.atleast_2d(point)
    theta = np.linspace(0, 1, nb_output)*2*math.pi #angles
    theta = theta.tolist()
    radius = 1/intenisty*scale #5% of the basin volume
    output_points = point + radius*np.array([np.cos(theta), np.sin(theta)]).T
    return output_points

# print(poisson_pp.points.shape)
# a = poisson_pp.points[0]
# b  = starting_points(a, intenisty=800)
# plt.plot(b[:,0], b[:,1], "b.")
# plt.plot(a[0], a[1], "r.")

In [ ]:
def trajectory(x, points, intensity, stepsize, t_stop):
    x_trajectory= [x]
    for t in range(t_stop):
        x_trajectory.append(x_trajectory[t] + stepsize*force_base(x_trajectory[t], points, intensity=intensity) )
    return np.vstack(x_trajectory)

# nb_output=15
# a = poisson_pp.points
# fig, ax= plt.subplots(figsize=(4,4))
# for k in range(a.shape[0]):
#     a_k = a[k]
#     b  = starting_points(a_k, intenisty=estimated_intensity,nb_output=nb_output) 
#     color = np.random.rand(3,)
#     for i in range(nb_output):
#         c = trajectory(b[i], a, estimated_intensity, stepsize=0.0001, t_stop=800)
#         ax.plot(c[:, 0], c[:, 1], c=color) 
#     ax.scatter(b[:,0], b[:,1], color=color, s=0.5)
#     ax.plot(a_k[0], a_k[1], "k*")  
# poisson_pp.plot(axis=ax)
# plt.show()


In [ ]:
d = 2
r = 5
simu_window = BallWindow(center=[0]*d, radius=r)
intensity = 1

# list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp = poisson.generate_point_pattern(window=simu_window, seed=120) 
epsilon = epsilon_critical(d=d, intensity=intensity)
print("Epsilon=", epsilon, ", Nb_points=", poisson_pp.points.shape[0])
estimated_intensity =poisson_pp.points.shape[0]/simu_window.volume
print("Intensity=", intensity, ", Estimated intensity= ", intensity)

In [ ]:

nb_points = 18 #number of points around each star
stepsize = 0.001 #stepsize of the discetization scheme of the flow curves
t_stop = 900 #stop time
fig, ax= plt.subplots(figsize=(4,4))
stars = poisson_pp.points #star for the gravitational allocation
for k in range(stars.shape[0]):
    points = starting_points(stars[k], 
                          intenisty=estimated_intensity,
                          nb_output=nb_points)  #starting points around stars_k
    color = np.random.rand(3,)*0.7 #color of the basin of stars_k
    for i in range(nb_points):
        points_t = trajectory(points[i], stars, estimated_intensity, stepsize=stepsize, t_stop=t_stop) #positions of point_i w.r.t. time
        ax.plot(points_t[:, 0], points_t[:, 1], c=color, linewidth=1, zorder=0, rasterized=True) 
ax.scatter(stars[:,0], stars[:, 1], color="k", s=5, zorder=5, label="PPP") 
ax.set_xticks([])
ax.set_yticks([])
plt.legend()
plt.tight_layout()
plt.savefig("gravitational_allocation_to_poisson.pdf")
plt.show()


In [ ]:
# with end points
nb_points = 18 #number of points around each star
stepsize = 0.001 #stepsize of the discetization scheme of the flow curves
t_stop = 1000 #stop time
fig, ax= plt.subplots(figsize=(4,4))
stars = poisson_pp.points #star for the gravitational allocation
for k in range(stars.shape[0]):
    points = starting_points(stars[k], 
                          intenisty=estimated_intensity,
                          nb_output=nb_points)  #starting points around stars_k
    color = np.random.rand(3,)*0.7 #color of the basin of stars_k
    for i in range(nb_points):
        points_t = trajectory(points[i], stars, estimated_intensity, stepsize=stepsize, t_stop=t_stop) #positions of point_i w.r.t. time
        ax.plot(points_t[:, 0], points_t[:, 1], c=color, linewidth=1, zorder=0, rasterized=True) 
        ax.scatter(points_t[-1, 0], points_t[-1, 1], color="b", s=2, zorder=10) #end points
ax.scatter(stars[:,0], stars[:, 1], color="k", s=5, zorder=5, label="PPP") 
ax.set_xticks([])
ax.set_yticks([])
plt.legend()
plt.tight_layout()
plt.savefig("gravitational_allocation_to_poisson_with_end_points.pdf")
plt.show()


# 2- Recursive repuslion of RPPP

In [ ]:
d = 2
r = 20
simu_window = BallWindow(center=[0]*d, radius=r)
intensity = 1
# list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp = poisson.generate_point_pattern(window=simu_window, seed=1) 
print("Epsilon=", epsilon, ", Nb_points=", poisson_pp.points.shape[0])


In [ ]:
t = [10, 30, 50, 80, 100, 130, 150, 180, 200]

# repelled pp
rpp = RepelledPointProcess(poisson_pp)
repelled = rpp.repelled_point_pattern()
epsilon = epsilon_critical(d=d, intensity=intensity)
print("Epsilon=", epsilon)
for t_stop in t:
    repelled_t = rpp.repelled_point_pattern(stop_time=t_stop)
    repelled_inverse_t = rpp.repelled_point_pattern(epsilon=-epsilon, stop_time=t_stop)
    print("t=", t_stop)
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    simu_window.plot(axis=ax[0], color="grey")
    poisson_pp.plot(axis=ax[0], c="g", s=0.1, label=r"$t=0$", rasterized=True)
    repelled_inverse_t.plot(axis=ax[0], window=simu_window, c="b", s=0.1, marker="*", label=r"$t=%d, \varepsilon = -\varepsilon_0$"%t_stop, rasterized=True)
    ax[0].legend()
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    simu_window.plot(axis=ax[1], color="grey")
    poisson_pp.plot(axis=ax[1], c="g", s=0.1, label=r"$t=0$", rasterized=True)
    repelled_t.plot(axis=ax[1], c="b", s=0.1, label=r"$t=%d, \varepsilon=\varepsilon_0$"%t_stop, rasterized=True)
    ax[1].legend()
    ax[1].set_xticks([])
    ax[1].set_yticks([])
    plt.tight_layout()
    plt.savefig("extreme_repelled_%d.pdf" %t_stop)
    plt.show()
